In [1]:
import tensorflow as tf

In [2]:
#verificar se GPU está ativa
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
!nvidia-smi

Sat May  8 22:33:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Exercícios - Conceitos fundamentais de Deep Learning e CNN utilizando MNIST e Keras**



## Revisando MLP e entendendo conceitos básicos




In [4]:
import tensorflow as tf 
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, GlobalAveragePooling2D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adadelta

import numpy as np

In [5]:
#Parâmetros de treinamento
batch_size = 32
epochs = 10
num_classes = 10

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [8]:
x_train[0, :, :]

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0

In [9]:
y_train[0]

5

In [10]:
x_test.shape

(10000, 28, 28)

In [11]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

01. O que o shape dessas arrays nos indicam sobre os dados? Ex: o que 60000 quer dizer e assim em diante..

O primeiro é o número de imagens do dataset, o segundo e terceiro são as dimensões das imagens.

In [12]:
#preprocessamento
x_train /= 255
x_test /= 255

In [13]:
x_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

02. Qual o objetivo desse preprocessamento? O que isso impacta no treinamento da rede?



O preprocessamento serve para normalizar os dados em valores entre 0 e 1, para adequar eles a função de ativação

In [14]:
#categorizando label de saída (não obrigatório, mas a melhor prática) 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [15]:
y_test.shape

(10000, 10)

In [16]:
y_test[1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

3. Copie o código abaixo, substituindo os 'x' pelo paramêtro correto. Cole na célula abaixo e execute o treinamento



```
model = Sequential()


model.add(x(input_shape=(784,)))
model.add(tf.keras.layers.Dense(256, input_dim=784, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.x(x, activation=x))


model.compile(loss='x',
            optimizer=x(1e-4),
            metrics=['accuracy'])

model.summary()


# Treinamento do modelo 
history = model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_data=(x, x))``````



In [17]:
model = Sequential()
 
model.add(Flatten(input_shape=(28, 28))) #shape das imagens
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax')) 
 
model.compile(loss='categorical_crossentropy', 
            optimizer=Adam(1e-4),
            metrics=['accuracy'])
 
model.summary()
 
# Treinamento do modelo 
history = model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_data=(x_test, y_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 8s 2ms/step - loss: 0.8589 - accuracy: 0.7690 - val_loss: 0.2287 - val_accuracy: 0.9364
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2125 - accuracy: 0.9397 - 

4. Como está o treinamento até agora? Está na hora de pararmos ou podemos treinar por mais épocas?

Não há necessidade de treinar por mais épocas pois não houve ganho de performance notável com mais épocas.

## Introdução a CNN



In [18]:
from tensorflow.keras import layers

In [19]:
#Parâmetros de treinamento
batch_size = 32
epochs = 10
num_classes = 10
num_filters_1 = 64
num_filters_2 = 32
num_filters_3 = 32
kernel_size = (3,3)
pool_size = (3,3)
activation = "relu"


In [20]:
def load_data_and_preprocess():
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_train /= 255
  X_test /= 255

  X_train = np.expand_dims(X_train, axis=-1) #Adicionando um canal a nossa imagem (28,28,1)
  X_test = np.expand_dims(X_test, axis=-1)

  #categorizando label de saída (não obrigatório, mas a melhor prática) 
  y_train = tf.keras.utils.to_categorical(y_train, num_classes)
  y_test = tf.keras.utils.to_categorical(y_test, num_classes)

  return X_train, y_train, X_test, y_test



In [21]:
def build_model():

  inputs = layers.Input(shape=(28, 28, 1))
  inputs_2 = layers.Input(shape=(30,30,1))

  x = layers.Conv2D(filters=num_filters_1, kernel_size=kernel_size, activation=activation)(inputs)
  x = layers.MaxPooling2D(pool_size=pool_size)(x)
  x = layers.Conv2D(filters=num_filters_2, kernel_size=kernel_size, activation=activation)(x)

  #x = layers.MaxPooling2D(pool_size=pool_size)(x)
  #x = layers.Conv2D(filters=num_filters_3, kernel_size=kernel_size, activation=activation)(x)
  #x = layers.MaxPooling2D(pool_size=pool_size)(x)

  x = layers.Flatten()(x)

  outputs = layers.Dense(num_classes, activation="softmax")(x)
  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(optimizer=Adam(learning_rate=2e-4), loss="categorical_crossentropy", metrics=['accuracy'])
  return model


In [22]:
X_train, y_train, X_test, y_test = load_data_and_preprocess()

In [23]:
X_train.shape

(60000, 28, 28, 1)

In [24]:
def training(model, x_train, y_train, x_test, y_test):
  history = model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_data=(x_test, y_test))
  return history

In [25]:
X_train, y_train, X_test, y_test = load_data_and_preprocess()

In [26]:
model = build_model()

In [27]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 32)          18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                11530     
Total params: 30,634
Trainable params: 30,634
Non-trainable params: 0
_________________________________________________________

In [28]:
h = training(model, X_train, y_train, X_test, y_test)

Epoch 1/10
1875/1875 [==============================] - 35s 3ms/step - loss: 0.7794 - accuracy: 0.7885 - val_loss: 0.1467 - val_accuracy: 0.9594
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1390 - accuracy: 0.9599 - val_loss: 0.0855 - val_accuracy: 0.9743
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0915 - accuracy: 0.9736 - val_loss: 0.0610 - val_accuracy: 0.9810
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0728 - accuracy: 0.9784 - val_loss: 0.0523 - val_accuracy: 0.9832
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0587 - accuracy: 0.9827 - val_loss: 0.0492 - val_accuracy: 0.9837
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0525 - accuracy: 0.9839 - val_loss: 0.0430 - val_accuracy: 0.9862
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0449 - accuracy: 0.9863 - val_loss: 0.0453 - val_accuracy

1. Compare as primeiras 10 épocas desse treinamento com o da rede MLP. O que você consegue concluir sobre a performance? Por que?





A acurácia foi um pouco maior e o loss foi um pouco menor ao longo do treinamento CNN, em comparação ao MLP.

2. Na função **build_model()**, adicione mais uma camada Conv2d e mais outra MaxPool e tente executar o treinamento. Foi possível? Explique o porquê.

Não foi possível, pois as dimensões das camadas ficaram de tamanhos diferentes.

3. Baseado no seguinte summary, construa mesma arquitetura e execute o treinamento.

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   

input_10 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 24, 24, 64)        73792     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 6, 6, 32)          18464     
_________________________________________________________________
flatten_18 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 10)                11530  
   
Total params: 105,066
Trainable params: 105,066
Non-trainable params: 0

In [29]:
def build_model():
    inputs = layers.Input(shape=(28, 28, 1))

    x = layers.Conv2D(filters=128, kernel_size=kernel_size, activation=activation)(inputs)
    x = layers.Conv2D(filters=num_filters_1, kernel_size=kernel_size, activation=activation)(x)
    x = layers.MaxPooling2D(pool_size=pool_size)(x)
    x = layers.Conv2D(filters=num_filters_2, kernel_size=kernel_size, activation=activation)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=Adam(learning_rate=2e-4), loss="categorical_crossentropy", metrics=['accuracy'])
    return model

In [30]:
model = build_model()

In [31]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 6, 32)          18464     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1153

In [32]:
X_train, y_train, X_test, y_test = load_data_and_preprocess()

In [33]:
h = training(model, X_train, y_train, X_test, y_test)

Epoch 1/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.5571 - accuracy: 0.8315 - val_loss: 0.0817 - val_accuracy: 0.9741
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0807 - accuracy: 0.9756 - val_loss: 0.0705 - val_accuracy: 0.9773
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0524 - accuracy: 0.9843 - val_loss: 0.0378 - val_accuracy: 0.9880
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0394 - accuracy: 0.9882 - val_loss: 0.0332 - val_accuracy: 0.9900
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0312 - accuracy: 0.9906 - val_loss: 0.0333 - val_accuracy: 0.9897
Epoch 6/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0250 - accuracy: 0.9923 - val_loss: 0.0282 - val_accuracy: 0.9910
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0211 - accuracy: 0.9931 - val_loss: 0.0308 - val_accuracy

4. Compare as primeiras 10 épocas desse treinamento com o da rede CNN anterior. O que você consegue concluir sobre a performance? Por que?

A performance caiu um pouco, mas a acurácia foi melhor, pois como foram adicionadas mais camadas, aumentou a quantidade de features também, melhorando a precisão do modelo.

5. Crie seu próprio modelo agora. Adicione camadas como desejar e altere os valores de treinamento.

In [34]:
batch_size = 64
epochs = 8
num_classes = 10

def build_model():
    inputs = layers.Input(shape=(28, 28, 1))

    x = layers.Conv2D(filters=128, kernel_size=kernel_size, activation=activation)(inputs)
    x = layers.Conv2D(filters=num_filters_1, kernel_size=kernel_size, activation=activation)(x)
    x = layers.Conv2D(filters=num_filters_2, kernel_size=kernel_size, activation=activation)(x)
    x = layers.Conv2D(filters=16, kernel_size=kernel_size, activation=activation)(x)
    x = layers.Flatten()(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=RMSprop(learning_rate=2e-4), loss="categorical_crossentropy", metrics=['accuracy'])
    return model

In [35]:
model = build_model()
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        73792     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 22, 22, 32)        18464     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 20, 16)        4624      
_________________________________________________________________
flatten_3 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                6401

In [36]:
X_train, y_train, X_test, y_test = load_data_and_preprocess()

In [37]:
h = training(model, X_train, y_train, X_test, y_test)

Epoch 1/8
938/938 [==============================] - 10s 10ms/step - loss: 0.4430 - accuracy: 0.8685 - val_loss: 0.0829 - val_accuracy: 0.9745
Epoch 2/8
938/938 [==============================] - 9s 10ms/step - loss: 0.0803 - accuracy: 0.9762 - val_loss: 0.0544 - val_accuracy: 0.9833
Epoch 3/8
938/938 [==============================] - 9s 10ms/step - loss: 0.0569 - accuracy: 0.9827 - val_loss: 0.0425 - val_accuracy: 0.9858
Epoch 4/8
938/938 [==============================] - 9s 10ms/step - loss: 0.0464 - accuracy: 0.9874 - val_loss: 0.0381 - val_accuracy: 0.9881
Epoch 5/8
938/938 [==============================] - 9s 10ms/step - loss: 0.0371 - accuracy: 0.9890 - val_loss: 0.0359 - val_accuracy: 0.9884
Epoch 6/8
938/938 [==============================] - 9s 10ms/step - loss: 0.0326 - accuracy: 0.9902 - val_loss: 0.0340 - val_accuracy: 0.9885
Epoch 7/8
938/938 [==============================] - 9s 10ms/step - loss: 0.0290 - accuracy: 0.9915 - val_loss: 0.0333 - val_accuracy: 0.9890
Epoch